In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/kaggle/severstal'
import sys
sys.path.append('lib')

In [ ]:
!pip uninstall -y opencv-python-headless 
!pip install -q opencv-python-headless==4.1.2.30

### Installatioin

In [ ]:
!pip install -q catalyst
!pip install -q -U albumentations
# !pip install -q tensorboardX
# !pip install -q hydra_slayer
!pip install -q segmentation_models_pytorch
!pip install -q -U pyyaml

### Import

In [ ]:
import numpy as np
import pandas as pd
import random, os, cv2
import matplotlib.pyplot as plt
from datetime import datetime
import yaml

import catalyst
from catalyst.contrib.losses import DiceLoss, FocalLossBinary
from catalyst.contrib.losses import LovaszLossMultiClass, FocalLossMultiClass
from catalyst.dl import CriterionCallback, MetricAggregationCallback
from catalyst.dl import BatchTransformCallback, DiceCallback, BatchMetricCallback
from catalyst.dl import SupervisedRunner
from catalyst.callbacks import SchedulerCallback

from segmentation_models_pytorch.losses import LovaszLoss, FocalLoss
from segmentation_models_pytorch import Unet, UnetPlusPlus, FPN, PSPNet

import torch
import torch.nn as nn
import torch.nn.functional as F

from commons import *
from catalyst_extension import CustomCheckpointCallback

### Configuration

In [ ]:
def provide_reproducibility():
    seed = 42
    torch.backends.cudnn.benchmark = False
    # torch.use_deterministic_algorithms(True)    
    random.seed(seed)
    np.random.seed(seed)    
    torch.manual_seed(seed)

provide_reproducibility()

In [ ]:
with open('config/segmentation_single_channel.yml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
config
logdir = config['checkpoint']['dir']

### Data loaders

In [ ]:
loaders = create_loaders(**config['dataset'])

### Model

In [ ]:
model = FPN('efficientnet-b5', encoder_weights='imagenet', in_channels=3, 
            classes=config['dataset']['num_classes'])

### Optimizer

In [ ]:
learning_rate = 5e-3
encoder_learning_rate = learning_rate / 10
weight_decay = 0.0003
layerwise_params = {
    "encoder*": dict(lr=learning_rate / 10, weight_decay=weight_decay / 10)
}
model_params = catalyst.utils.process_model_params(model, layerwise_params=layerwise_params)
base_optimizer = torch.optim.RAdam(model_params, lr=learning_rate, 
                                   weight_decay=weight_decay)
optimizer = catalyst.contrib.optimizers.Lookahead(base_optimizer)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                       factor=0.25, patience=10)

### Catalyst configuration

In [ ]:

criterion = {
    "dice": DiceLossPerImage(),
    'bce': nn.BCEWithLogitsLoss(),
}

callbacks = [
    BatchTransformCallback(
        scope="on_batch_end",
        transform=torch.sigmoid,
        input_key="logits",
        output_key="outputs",
    ),
    CriterionCallback("outputs", "mask_onehot", "loss_dice", criterion_key="dice"),
    CriterionCallback("logits", "mask_onehot", "loss_bce", criterion_key="bce"),

    MetricAggregationCallback(
        "loss",
        mode="weighted_sum",
        metrics={'loss_bce': 1.0, 'loss_dice': 1.0},
    ),
    BatchMetricCallback(DiceMetricPerImage(), 'outputs', 'mask_onehot', 
        log_on_batch=False),
    CustomCheckpointCallback(logdir=f'{logdir}/checkpoints', loader_key='valid', 
                             metric_key='dice_pi', minimize=False),
    SchedulerCallback(loader_key='valid', metric_key='loss'),
]

### Training

In [ ]:
runner = SupervisedRunner(input_key="image", output_key="logits")
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    callbacks=callbacks,
    logdir=logdir,
    num_epochs=100,
    verbose=True,
    resume=config['checkpoint']['resume']
)